# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [18]:
# import numpy and pandas
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.stats import t, ttest_1samp
from statsmodels.stats.proportion import proportions_ztest


# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [2]:
# Run this code:

salaries = pd.read_csv('../data/Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [3]:
# Your code here:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [4]:
# Your code here:
missing_values = salaries.isnull().sum()
missing_values

Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [5]:
# Your code here:
employee_counts = salaries['Salary or Hourly'].value_counts()
employee_counts

Salary    25161
Hourly     8022
Name: Salary or Hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [6]:
employee_counts_df = salaries['Department'].value_counts().to_frame().reset_index()

# Rename the columns
employee_counts_df.columns = ['Department', 'Employee Count']



# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [7]:

# Extract the hourly wage column for hourly workers
hourly_wages = salaries.loc[salaries['Salary or Hourly'] == 'Hourly', 'Hourly Rate']

# Perform one-sample t-test
t_stat, p_val = ttest_1samp(hourly_wages, 30)

# Compute the confidence interval
alpha = 0.05
n = len(hourly_wages)
df = n - 1
t_crit = abs(t.ppf(alpha/2, df))
stderr = hourly_wages.std() / np.sqrt(n)
margin_err = t_crit * stderr
ci_lower = hourly_wages.mean() - margin_err
ci_upper = hourly_wages.mean() + margin_err

# Print the results
print('t-statistic:', t_stat)
print('p-value:', p_val)
print('95% confidence interval:', (ci_lower, ci_upper))


t-statistic: 20.6198057854942
p-value: 4.3230240486229894e-92
95% confidence interval: (32.52345834488425, 33.05365708767623)


We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [8]:
# Extract the salary column for police officers
police_salaries = salaries.loc[(salaries['Department'] == 'POLICE') & (salaries['Salary or Hourly'] == 'Salary'), 'Annual Salary']

# Perform one-sample t-test
t_stat, p_val = ttest_1samp(police_salaries, 86000, alternative='greater')

# Compute the 95% confidence interval
alpha = 0.05
n = len(police_salaries)
df = n - 1
t_crit = t.ppf(1 - alpha, df)
stderr = police_salaries.std() / np.sqrt(n)
margin_err = t_crit * stderr
ci_lower = police_salaries.mean() - margin_err
ci_upper = police_salaries.mean() + margin_err

# Print the results
print('t-statistic:', t_stat)
print('p-value:', p_val/2) # divide by 2 for a one-sided test
print('95% confidence interval:', (ci_lower, ci_upper))


t-statistic: 3.081997005712994
p-value: 0.0005150850887741284
95% confidence interval: (86226.79842892641, 86746.03057734037)


Using the `crosstab` function, find the department that has the most hourly workers. 

In [9]:
# Create a contingency table of department and employment status
dept_emp_table = pd.crosstab(salaries['Department'], salaries['Salary or Hourly'])

# Extract the hourly worker counts for each department
hourly_counts = dept_emp_table['Hourly']

# Find the department with the most hourly workers
most_hourly_dept = hourly_counts.idxmax()

# Print the result
print("The department with the most hourly workers is", most_hourly_dept)


The department with the most hourly workers is STREETS & SAN


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [10]:
dept_emp_table = pd.crosstab(salaries['Department'], salaries['Salary or Hourly'])
most_hourly_dept = dept_emp_table['Hourly'].idxmax()
hourly_wages = salaries.loc[(salaries['Department'] == most_hourly_dept) & (salaries['Salary or Hourly'] == 'Hourly'), 'Hourly Rate']
null_hypothesis = 35

# Calculate the t-value and p-value of the one-sided t-test
t_stat, p_val = ttest_1samp(hourly_wages, null_hypothesis)
alpha = 0.05
if p_val/2 < alpha and t_stat < 0:
    print("Reject the null hypothesis. The hourly wage is less than $35/hour.")
else:
    print("Do not reject the null hypothesis. The hourly wage is not less than $35/hour.")


Reject the null hypothesis. The hourly wage is less than $35/hour.


# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [11]:
# calculate sample mean and standard error
mean = hourly_wages.mean()
std_err = hourly_wages.std() / np.sqrt(len(hourly_wages))

# set confidence level and degrees of freedom
alpha = 0.05
df = len(hourly_wages) - 1

# calculate t-value for given confidence level and degrees of freedom
t = stats.t.ppf(1 - alpha/2, df)

# calculate lower and upper bounds of confidence interval
lower_bound = mean - t * std_err
upper_bound = mean + t * std_err

# print results
print(f"Hourly wage confidence interval (95%): ({lower_bound:.2f}, {upper_bound:.2f})")


Hourly wage confidence interval (95%): (33.47, 33.99)


Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [16]:
# Filter the dataset to include only salaried employees in the police department
police_salaries_interval = salaries[(salaries['Department'] == 'POLICE') & (salaries['Salary or Hourly'] == 'Salary')]

# Calculate the mean and standard error of their salaries
mean_salary = police_salaries_interval['Annual Salary'].mean()
std_error = stats.sem(police_salaries_interval['Annual Salary'])

# Calculate the t-value and margin of error for the confidence interval
alpha = 0.05
n = len(police_salaries_interval)
df = n - 1
t_dist = t(df)
t_crit_value = abs(t_dist.ppf(alpha/2))
margin_err = t_crit_value * std_error

# Construct the confidence interval
lower_bound = mean_salary - margin_err
upper_bound = mean_salary + margin_err

print(f"The 95% confidence interval for all salaried employees in the police department is ({lower_bound:.2f}, {upper_bound:.2f})")


The 95% confidence interval for all salaried employees in the police department is (86177.06, 86795.77)


# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [17]:
# Your code here:

# Calculate the number of hourly and salaried workers
n_hourly = salaries['Salary or Hourly'].value_counts()['Hourly']
n_salaried = salaries['Salary or Hourly'].value_counts()['Salary']

# Set the null hypothesis proportion
null_prop = 0.25

# Calculate the sample proportion of hourly workers
sample_prop = n_hourly / (n_hourly + n_salaried)

# Perform the hypothesis test
stat, pval = proportions_ztest(count=n_hourly, nobs=n_hourly + n_salaried, value=null_prop)

# Check if the p-value is less than the significance level (0.05)
if pval < 0.05:
    print("Reject null hypothesis")
else:
    print("Fail to reject null hypothesis")


Reject null hypothesis
